In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Conv2D, Input, AveragePooling2D, Activation, GlobalAveragePooling2D, Concatenate, BatchNormalization, MaxPooling2D, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers.merge import concatenate
from tensorflow.keras.models import load_model

num_classes = 200

img_height, img_width = 32, 32

img_channels = 3

#Import Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -qq 'tiny-imagenet-200.zip'

--2021-12-21 13:09:16--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  10.1MB/s    in 19s     

2021-12-21 13:09:35 (12.3 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [14]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(5)

,File,Class
0,val_0.JPEG,n03444034
1,val_1.JPEG,n04067472
2,val_2.JPEG,n04070727
3,val_3.JPEG,n02808440
4,val_4.JPEG,n02808440


In [15]:
train_datagen = ImageDataGenerator(rescale= 1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(32, 32), color_mode='rgb', 
                                                    batch_size=256, class_mode='categorical', shuffle=True, seed=42)

validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=256, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


## Model

In [16]:
def space_to_depth(x):
    return tf.nn.space_to_depth(x, block_size=2)

In [17]:
input = Input(shape=(None, None, img_channels))

# Block 1

layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
layer1 = BatchNormalization(name='norm_1')(layer1)
layer1 = Activation("relu")(layer1)

layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)
layer2 = BatchNormalization(name='norm_2')(layer2)
layer2 = Activation("relu")(layer2)

layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer2)
layer3 = BatchNormalization(name='norm_3')(layer3)
layer3 = Activation("relu")(layer3)

layer4 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer3)
layer4 = BatchNormalization(name='norm_4')(layer4)
layer4 = Activation("relu")(layer4)

layer5 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(layer4)
layer5 = BatchNormalization(name='norm_5')(layer5)
layer5 = Activation("relu")(layer5)

layer6 = MaxPooling2D(pool_size=(2, 2))(layer5)

skip_connection_1 = layer6

# Block 2

layer7 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_7', use_bias=False)(layer6)
layer7 = BatchNormalization(name='norm_7')(layer7)
layer7 = Activation("relu")(layer7)

layer8 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(layer7)
layer8 = BatchNormalization(name='norm_8')(layer8)
layer8 = Activation("relu")(layer8)

layer9 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(layer8)
layer9 = BatchNormalization(name='norm_9')(layer9)
layer9 = Activation("relu")(layer9)

layer10 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_10', use_bias=False)(layer9)
layer10 = BatchNormalization(name='norm_10')(layer10)
layer10 = Activation("relu")(layer10)

layer11 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(layer10)
layer11 = BatchNormalization(name='norm_11')(layer11)
layer11 = Activation("relu")(layer11)

layer12 = MaxPooling2D(pool_size=(2, 2))(layer11)

skip_connection_1 = Lambda(space_to_depth)(skip_connection_1)

layer13 = concatenate([skip_connection_1, layer12])

skip_connection_2 = layer13

# Block 3

layer14 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(layer13)
layer14 = BatchNormalization(name='norm_14')(layer14)
layer14 = Activation("relu")(layer14)

layer15 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_15', use_bias=False)(layer14)
layer15 = BatchNormalization(name='norm_15')(layer15)
layer15 = Activation("relu")(layer15)

layer16 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(layer15)
layer16 = BatchNormalization(name='norm_16')(layer16)
layer16 = Activation("relu")(layer16)

layer17 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_17', use_bias=False)(layer16)
layer17 = BatchNormalization(name='norm_17')(layer17)
layer17 = Activation("relu")(layer17)

layer18 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(layer17)
layer18 = BatchNormalization(name='norm_18')(layer18)
layer18 = Activation("relu")(layer18)

layer19 = MaxPooling2D(pool_size=(2, 2))(layer18)

skip_connection_2 = Lambda(space_to_depth)(skip_connection_2)

layer20 = concatenate([skip_connection_2, layer19])

layer21 = Conv2D(num_classes, (1,1), name='conv_21', use_bias=False)(layer20)
layer21 = BatchNormalization(name='norm_21')(layer21)

layer22 = GlobalAveragePooling2D(data_format=None)(layer21)
layer22 = tf.keras.layers.Dense(1024)(layer22)
layer22 = BatchNormalization()(layer22)
layer22 = Activation('relu')(layer22)

layer23 = tf.keras.layers.Dense(512)(layer22)
layer23 = BatchNormalization()(layer23)
layer23 = Activation('relu')(layer23)

layer24 = tf.keras.layers.Dropout(0.25)(layer23)

layer25 = tf.keras.layers.Dense(200,activation='softmax')(layer24)

output = layer25

In [18]:
model = tf.keras.models.Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv_1 (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 norm_1 (BatchNormalization)    (None, None, None,   128         ['conv_1[0][0]']                 
                                32)                                                           

In [19]:
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Training

## Epoch 1-20

In [20]:
history1 = model.fit(train_generator, 
                     epochs=20, 
                     steps_per_epoch=200, 
                     validation_data=validation_generator, 
                     callbacks=[lr_reducer])

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_1.h5') 

df = pd.DataFrame(history1.history)
df.to_csv('mhistory1.csv')
from google.colab import files
files.download('mhistory1.csv')

Epoch 1/20
200/200 [==============================] - 131s 448ms/step - loss: 4.8070 - accuracy: 0.0461 - val_loss: 6.4008 - val_accuracy: 0.0086 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 84s 421ms/step - loss: 4.1854 - accuracy: 0.1061 - val_loss: 5.5262 - val_accuracy: 0.0359 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 84s 422ms/step - loss: 3.8517 - accuracy: 0.1493 - val_loss: 4.8059 - val_accuracy: 0.0726 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 84s 418ms/step - loss: 3.6241 - accuracy: 0.1820 - val_loss: 5.2527 - val_accuracy: 0.0564 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 83s 416ms/step - loss: 3.4571 - accuracy: 0.2085 - val_loss: 5.2644 - val_accuracy: 0.0600 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 83s 416ms/step - loss: 3.3082 - accuracy: 0.2356 - val_loss: 4.0650 - val_accuracy: 0.1295 - lr: 0.0010
Epoch 7/20
200/200 [==============================]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=64, class_mode='categorical', shuffle=True, seed=42)

validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=64, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [22]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_1.h5')

history2 = model.fit(train_generator, epochs=15, steps_per_epoch=200, validation_data=validation_generator, callbacks=[lr_reducer])

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_2.h5') 

df = pd.DataFrame(history2.history)
df.to_csv('mhistory2.csv')
files.download('mhistory2.csv')

Epoch 1/15
200/200 [==============================] - 83s 410ms/step - loss: 2.8647 - accuracy: 0.3199 - val_loss: 3.6754 - val_accuracy: 0.2340 - lr: 3.1623e-04
Epoch 2/15
200/200 [==============================] - 82s 409ms/step - loss: 2.5494 - accuracy: 0.3730 - val_loss: 3.0989 - val_accuracy: 0.2867 - lr: 3.1623e-04
Epoch 3/15
200/200 [==============================] - 81s 408ms/step - loss: 2.4496 - accuracy: 0.3985 - val_loss: 2.8826 - val_accuracy: 0.3229 - lr: 3.1623e-04
Epoch 4/15
200/200 [==============================] - 82s 408ms/step - loss: 2.3791 - accuracy: 0.4052 - val_loss: 2.8447 - val_accuracy: 0.3332 - lr: 3.1623e-04
Epoch 5/15
200/200 [==============================] - 82s 408ms/step - loss: 2.3333 - accuracy: 0.4183 - val_loss: 2.8363 - val_accuracy: 0.3287 - lr: 3.1623e-04
Epoch 6/15
200/200 [==============================] - 81s 407ms/step - loss: 2.3050 - accuracy: 0.4306 - val_loss: 3.2882 - val_accuracy: 0.2952 - lr: 3.1623e-04
Epoch 7/15
200/200 [========

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_2.h5')

history3 = model.fit(train_generator, epochs=15, steps_per_epoch=200, validation_data=validation_generator)

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_3.h5')

df = pd.DataFrame(history3.history)
df.to_csv('mhistory3.csv')
files.download('mhistory3.csv')

Epoch 1/15
200/200 [==============================] - 83s 410ms/step - loss: 2.0286 - accuracy: 0.4862 - val_loss: 2.5800 - val_accuracy: 0.3841
Epoch 2/15
200/200 [==============================] - 82s 409ms/step - loss: 2.0295 - accuracy: 0.4855 - val_loss: 3.0191 - val_accuracy: 0.3224
Epoch 3/15
200/200 [==============================] - 82s 408ms/step - loss: 2.0093 - accuracy: 0.4817 - val_loss: 2.7257 - val_accuracy: 0.3569
Epoch 4/15
200/200 [==============================] - 82s 408ms/step - loss: 2.0215 - accuracy: 0.4873 - val_loss: 2.6556 - val_accuracy: 0.3737
Epoch 5/15
200/200 [==============================] - 82s 408ms/step - loss: 1.9476 - accuracy: 0.5066 - val_loss: 2.7320 - val_accuracy: 0.3536
Epoch 6/15
200/200 [==============================] - 82s 409ms/step - loss: 1.9574 - accuracy: 0.4985 - val_loss: 2.7462 - val_accuracy: 0.3570
Epoch 7/15
200/200 [==============================] - 82s 408ms/step - loss: 1.9443 - accuracy: 0.5025 - val_loss: 2.6417 - val_ac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(16, 16), color_mode='rgb', 
                                                    batch_size=256, class_mode='categorical', shuffle=True, seed=42)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=256, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [25]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_3.h5')

history4 = model.fit(train_generator, 
                     epochs=10, 
                     steps_per_epoch=200, 
                     validation_data=validation_generator)

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_4.h5')

df = pd.DataFrame(history4.history)
df.to_csv('mhistory4.csv')
from google.colab import files
files.download('mhistory4.csv')

Epoch 1/10
200/200 [==============================] - 42s 199ms/step - loss: 3.5812 - accuracy: 0.2219 - val_loss: 5.3282 - val_accuracy: 0.0762
Epoch 2/10
200/200 [==============================] - 39s 197ms/step - loss: 3.0475 - accuracy: 0.2923 - val_loss: 5.1308 - val_accuracy: 0.0676
Epoch 3/10
200/200 [==============================] - 39s 194ms/step - loss: 2.8619 - accuracy: 0.3269 - val_loss: 5.2500 - val_accuracy: 0.0659
Epoch 4/10
200/200 [==============================] - 39s 193ms/step - loss: 2.7739 - accuracy: 0.3385 - val_loss: 5.7183 - val_accuracy: 0.0568
Epoch 5/10
200/200 [==============================] - 39s 194ms/step - loss: 2.6610 - accuracy: 0.3638 - val_loss: 5.1914 - val_accuracy: 0.0758
Epoch 6/10
200/200 [==============================] - 39s 194ms/step - loss: 2.5653 - accuracy: 0.3773 - val_loss: 5.5818 - val_accuracy: 0.0652
Epoch 7/10
200/200 [==============================] - 39s 194ms/step - loss: 2.4917 - accuracy: 0.3957 - val_loss: 6.1450 - val_ac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=64, class_mode='categorical', shuffle=True, seed=42)

validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=64, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [27]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_4.h5')

history5 = model.fit(train_generator, 
                     epochs=15, 
                     steps_per_epoch=200, 
                     validation_data=validation_generator)

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_5.h5')

df = pd.DataFrame(history5.history)
df.to_csv('mhistory5.csv')
from google.colab import files
files.download('mhistory5.csv')

Epoch 1/15
200/200 [==============================] - 83s 409ms/step - loss: 2.2601 - accuracy: 0.4266 - val_loss: 3.6999 - val_accuracy: 0.2624
Epoch 2/15
200/200 [==============================] - 81s 408ms/step - loss: 1.9967 - accuracy: 0.4886 - val_loss: 2.8426 - val_accuracy: 0.3464
Epoch 3/15
200/200 [==============================] - 81s 405ms/step - loss: 1.9167 - accuracy: 0.5036 - val_loss: 2.6545 - val_accuracy: 0.3753
Epoch 4/15
200/200 [==============================] - 81s 407ms/step - loss: 1.8511 - accuracy: 0.5229 - val_loss: 2.7971 - val_accuracy: 0.3526
Epoch 5/15
200/200 [==============================] - 81s 407ms/step - loss: 1.8056 - accuracy: 0.5302 - val_loss: 2.8082 - val_accuracy: 0.3545
Epoch 6/15
200/200 [==============================] - 81s 407ms/step - loss: 1.7736 - accuracy: 0.5380 - val_loss: 2.5184 - val_accuracy: 0.3994
Epoch 7/15
200/200 [==============================] - 81s 407ms/step - loss: 1.7638 - accuracy: 0.5461 - val_loss: 2.6956 - val_ac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Image Augmentation

In [28]:
from imgaug import augmenters as iaa

seq = iaa.SomeOf((0, None), [
                             iaa.Affine(scale=(0.5, 1.5)), 
                             iaa.Affine(rotate=20),
                             iaa.CoarseDropout((0.0, 0.2), 
                             size_percent=(0.05, 0.07)), 
                             iaa.AdditiveGaussianNoise(scale=0.05*255), 
                             iaa.CropAndPad(percent=(-0.25, 0.25))
     ])

train_datagen = ImageDataGenerator(preprocessing_function=seq.augment_image, rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

In [29]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=64, class_mode='categorical', shuffle=True, seed=42)

validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=64, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [30]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_5.h5')

history6 = model.fit(train_generator, 
                     epochs=30, 
                     steps_per_epoch=200, 
                     validation_data=validation_generator)

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_6.h5')

df = pd.DataFrame(history6.history)
df.to_csv('mhistory6.csv')
files.download('mhistory6.csv')

Epoch 1/30
200/200 [==============================] - 102s 504ms/step - loss: 3.3318 - accuracy: 0.2747 - val_loss: 4.3835 - val_accuracy: 0.2430
Epoch 2/30
200/200 [==============================] - 99s 494ms/step - loss: 2.9705 - accuracy: 0.3127 - val_loss: 3.3310 - val_accuracy: 0.2978
Epoch 3/30
200/200 [==============================] - 98s 490ms/step - loss: 2.8707 - accuracy: 0.3286 - val_loss: 2.9827 - val_accuracy: 0.3458
Epoch 4/30
200/200 [==============================] - 102s 509ms/step - loss: 2.8133 - accuracy: 0.3381 - val_loss: 2.8623 - val_accuracy: 0.3537
Epoch 5/30
200/200 [==============================] - 99s 492ms/step - loss: 2.7766 - accuracy: 0.3462 - val_loss: 2.9967 - val_accuracy: 0.3356
Epoch 6/30
200/200 [==============================] - 98s 490ms/step - loss: 2.7260 - accuracy: 0.3559 - val_loss: 2.9360 - val_accuracy: 0.3422
Epoch 7/30
200/200 [==============================] - 98s 489ms/step - loss: 2.7027 - accuracy: 0.3566 - val_loss: 2.9674 - val_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_6.h5')

history7 = model.fit(train_generator, 
                     epochs=30, 
                     steps_per_epoch=200, 
                     validation_data=validation_generator)

model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_7.h5')

df = pd.DataFrame(history7.history)
df.to_csv('mhistory7.csv')
files.download('mhistory7.csv')

Epoch 1/30
200/200 [==============================] - 99s 491ms/step - loss: 2.4519 - accuracy: 0.4040 - val_loss: 2.8877 - val_accuracy: 0.3529
Epoch 2/30
200/200 [==============================] - 98s 490ms/step - loss: 2.4292 - accuracy: 0.4118 - val_loss: 2.6619 - val_accuracy: 0.3885
Epoch 3/30
200/200 [==============================] - 98s 488ms/step - loss: 2.4031 - accuracy: 0.4185 - val_loss: 2.6930 - val_accuracy: 0.3799
Epoch 4/30
200/200 [==============================] - 98s 488ms/step - loss: 2.4278 - accuracy: 0.4120 - val_loss: 2.7189 - val_accuracy: 0.3852
Epoch 5/30
200/200 [==============================] - 98s 487ms/step - loss: 2.4103 - accuracy: 0.4146 - val_loss: 2.6303 - val_accuracy: 0.3923
Epoch 6/30
200/200 [==============================] - 98s 491ms/step - loss: 2.4402 - accuracy: 0.4096 - val_loss: 2.7210 - val_accuracy: 0.3815
Epoch 7/30
200/200 [==============================] - 98s 488ms/step - loss: 2.4228 - accuracy: 0.4115 - val_loss: 2.5411 - val_ac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Evaluation

In [32]:
valid_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=64, shuffle=True, seed=42)

Found 10000 validated image filenames belonging to 200 classes.


In [34]:
model = load_model('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Modified Models_1_1/Model1_7.h5')

validation_steps_per_epoch = np.math.ceil(validation_generator.samples / validation_generator.batch_size)

score = model.evaluate(validation_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

157/157 [==============================] - 18s 109ms/step - loss: 2.5779 - accuracy: 0.4052
Test loss: 2.5779263973236084
Test accuracy: 0.4052000045776367
